In [49]:
!pip install fastapi
!pip install tensorflow==2.15.0
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import pandas as pd
import numpy as np
from datetime import timedelta
from tensorflow.keras.models import load_model
import joblib


In [51]:

app = FastAPI()

# Caminhos para os arquivos salvos
model_path = 'C:/Users/ermlu/Downloads/lstm_cnn_attention_model.h5'
scaler_price_path = 'C:/Users/ermlu/Downloads/scaler_price.pkl'
scaler_timestamp_path = 'C:/Users/ermlu/Downloads/scaler_timestamp.pkl'
encoder_path = 'C:/Users/ermlu/Downloads/encoder.pkl'

# Carregar os modelos e transformadores salvos
scaler_price = joblib.load(scaler_price_path)
scaler_timestamp = joblib.load(scaler_timestamp_path)
encoder = joblib.load(encoder_path)
model = load_model(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    # Carregar os dados
    df_previsao = pd.read_csv(file.file)
    df_previsao['Data do pregão'] = pd.to_datetime(df_previsao['Data do pregão'])
    df_previsao['Data do pregão'] = df_previsao['Data do pregão'].map(pd.Timestamp.timestamp)
    df_previsao['Preço do último negócio normalizado'] = scaler_price.transform(df_previsao[['Preço do último negócio']])
    df_previsao['Data do pregão normalizado'] = scaler_timestamp.transform(df_previsao[['Data do pregão']])
    
    # One-Hot Encoding
    df_previsao['Código de negociação'] = df_previsao['Código de negociação'].astype(str)
    encoded_features = encoder.transform(df_previsao[['Código de negociação']])
    encoded_feature_names = encoder.get_feature_names_out(['Código de negociação'])
    df_encoded = pd.DataFrame(encoded_features.toarray(), columns=encoded_feature_names, index=df_previsao.index)
    df_final = pd.concat([df_previsao, df_encoded], axis=1)
    
    feature_columns = ['Preço do último negócio normalizado', 'Data do pregão normalizado'] + list(encoded_feature_names)
    data = df_final[feature_columns].values
    
    # Iniciar previsões
    start_date = df_previsao['Data do pregão'].max()
    acoes_interesse = ['VALE3', 'PETR4', 'ITUB4', 'BBDC4', 'B3SA3']
    results = []
    
    for day in range(30):
        current_date = pd.to_datetime(start_date, unit='s') + timedelta(days=day + 1)
        for acao in acoes_interesse:
            action_encoded = encoder.transform(pd.DataFrame([acao], columns=['Código de negociação'])).toarray()[0]
            new_row = np.concatenate([data[-1, :-len(acoes_interesse)], action_encoded])
            
            input_data = np.vstack([data[-59:, :], new_row.reshape(1, -1)])
            
            if input_data.shape[0] == 60:
                X = input_data.reshape(1, 60, -1)
                prediction = model.predict(X)
                predicted_price = scaler_price.inverse_transform(prediction.reshape(1, -1))[0][0]
                
                results.append({
                    'Data do pregão': (current_date + timedelta(days=day)).strftime('%Y-%m-%d'),
                    'Código de negociação': acao,
                    'Preço previsto': predicted_price
                })
                
                # Atualizar 'data' para incluir a nova previsão
                new_row[0] = prediction.flatten()[0]
                data = np.vstack([data, new_row])
    
    return JSONResponse(content={"results": results})